This notebook does the evaluation of the models. 

The modules used in this notebook are imported below. 

- The module utils_model contains auxiliary files for the model.
- The module network contains different neural network architectures.

In [4]:
from utils_model.data_helper import *
from utils_model.model_utils import *
from utils_model.visualization import *
from utils_model.process_bpm import *
from utils_model.eval_methods import *
from utils_model.data_prep_model import *
from network import LSTM as MyModel

from sklearn.model_selection import ParameterGrid 
import copy

## Load model

In [4]:
# Choose the paramters for the model
params = {
            'epochs':[1500],
            'batch_size':[24],
            'lr_scheduler':['plateau'],
            'units': [[90,60,30,1]],
            'loss':['custom'], 
            'metrics':[['r','rmse','mse']],
            'optimizer':['adam'], 
            'datasets': [['PURE']], 
            'datasets_only_test':[[]],
            'methods_list': [['gt','cpu_LGI','cpu_CHROM','cpu_POS','cpu_ICA']],  
            'landmarks':['combined','prio_all'],
            'fps' : [30],
            'win_secs':[10],
            'norm':['min_max'],
            'act_filter':['all'],
            'dropout':[.1],
            'K_fold':[5],
            'early_stopping':[True],
            'overlap':[0],
}

param_grid = ParameterGrid(params)
# This is in case we want to run only one experiment
param = param_grid[0]
param['samples'] = int(param['fps']*param['win_secs'])
param['n_methods'] = len(param['methods_list'])-1
landmarks = select_landmarks(param['landmarks'])
param['n_landmarks'] = len(landmarks)

In [ ]:
# Specify the paths to the trained model weights  
m1 = './kfoldpure/1/MODEL7~1.H5'
m2 = './kfoldpure/2/MODEL6~2.H5'
m3 = './kfoldpure/3/MODEL4~1.H5'  
m4 = './kfoldpure/4/MODEL3~3.H5' 
m5 = './kfoldpure/5/MODEL4~2.H5' 

# Store the paths in a list
weights = [m1, m2, m3, m4, m5]

# Choose the number of landmarks to be used based on the value of 'landmarks' in param
if param['landmarks']=='all':
    n_landmarks = 455
elif param['landmarks']=='forehead':
    n_landmarks = 10
elif param['landmarks']=='combined':
    n_landmarks = 30
else:
    # If 'landmarks' is not one of the expected values, print an error message
    print('CHECK LANDMARKS')

# Calculate the number of samples based on the frame rate and window size
param['samples'] = int(param['fps'] * param['win_secs'])

# Determine the number of methods to be used
n_methods = len(param['methods_list'])-1

# Choose the loss function
loss = choose_metric(param['loss'])

# Choose the metrics to be used for evaluation
metrics = []
for metric in param['metrics']: 
    metrics.append(choose_metric(metric))

# Choose the optimizer to be used
optimizer = choose_optimizer(param['optimizer'])


# Create an empty list to store the models
models = []

# Loop through the list of weights and load the corresponding models
for weight in weights:
    # Create an instance of the MyModel class with the chosen parameters
    model = MyModel(param['samples'], n_methods, n_landmarks, drop=param['dropout'], units=param['units'])

    # Compile the model with the chosen loss function, metrics, and optimizer
    model.compile(loss=CustomLoss(loss), metrics=metrics, optimizer=optimizer)

    # Build the model with the appropriate input shape and load the weights from the specified directory
    model.build(input_shape=(param['batch_size'], param['samples'], n_methods*n_landmarks))
    model.load_weights(weight)

    # Append the model to the list of models
    models.append(model)

## Evaluation

In [ ]:
fps = {
    'PURE': {'gt': 60, 'rPPG': param['fps']},
    'LGI-PPGI': {'gt': 60, 'rPPG': param['fps']},
    'MR-NIRP': {'gt': 60, 'rPPG': param['fps']}
        }

data_eval, fps = load_dataset('.\\pyVHR\\datasets',['PURE', 'LGI-PPGI', 'MR-NIRP'])
data_no_window = copy.deepcopy(data_eval)
data_eval = clean_landmarks(data_eval)
data_eval = resample_gt(data_eval)
data_eval = split_dataset_windows(data_eval, param['win_secs'], fps, overlap=0)
data_eval = resample_windows(data_eval,param['win_secs'],param['fps'])
data_eval = norm_windows(data_eval, mode=param['norm'],dim=3)
data_eval = clean_windowed_dataset(data_eval)
data_eval = hist_equalize(data_eval)


# Get the BPM 
data_bpm = load_bpm(data_eval)
data_bpm = resample_bpm(data_bpm,data_no_window)  
data_bpm = split_bpm_windows(data_bpm, param['win_secs'], fps, overlap=0)
data_bpm = resample_windows(data_bpm,param['win_secs'],param['fps'])

In [6]:
# Create the names splits for the 5-fold cross validation
# PURE dataset: names are 01,02,03,04,05,06,08,09,10
names_splits = [{'train': {'PURE': ['03', '04', '05', '06', '08', '09', '10']},
                'test': {'PURE': ['01', '02']}},
                {'train': {'PURE': ['01', '02', '05', '06', '08', '09', '10']},
                                'test': {'PURE': ['03', '04']}},
                {'train': {'PURE': ['01', '02', '03', '04', '08', '09', '10']},
                                'test': {'PURE': ['05', '06']}},
                {'train': {'PURE': ['01', '02', '03', '04', '05', '06', '10']},
                                'test': {'PURE': ['08', '09']}},
                {'train': {'PURE': ['01', '02', '03', '04', '05', '06', '08', '09']},
                                'test': {'PURE': ['10']}}]
 

In [ ]:
# Do the evaluation for the traditional methods
data_results = nested_dict(6, list)
data_results_methods = eval_methods(data_eval,data_results,fps=param['fps'],data_bpm=data_bpm)

### Individual model

In [ ]:
# Do just one evaluation of the split you have trained the model with
data_results,data_eval = eval_models(data_eval,param,[model],copy.deepcopy(data_results_methods),names_splits,split=0,fps=param['fps'],isprint=True,data_bpm=data_bpm)

In [ ]:
# For the case where I train with resting, I want to evaluate all the dataset with the split 0
# Evaluate with the split you have trained the model with all the splits
# The only part present in the train and test, will be the activity resting of the subjects used to train: ['cpi', 'david', 'felix', 'harun']
data_results,data_eval = eval_models(data_eval,param,[model,model,model,model,model],copy.deepcopy(data_results_methods),
                                    names_splits,fps=param['fps'],isprint=True,data_bpm=data_bpm)

In [ ]:
# for the other datasets that are not in the train/test set, we evaluate them with the best split
data_results,data_eval = eval_models(data_eval,param,[model],copy.deepcopy(data_results),names_splits,fps=param['fps'],isprint=False,
                                     datasets=['PURE','MR-NIRP'],split=0,data_bpm=data_bpm)

### CV

In [ ]:
# do evaluation for the datasets in the training
data_results,data_eval = eval_models(data_eval,param,models,copy.deepcopy(data_results_methods),names_splits,fps=param['fps'],isprint=True,data_bpm=data_bpm)


In [ ]:
# for the other datasets that are not in the train/test set, we evaluate them 
data_results,data_eval = eval_models(data_eval,param,models,data_results,names_splits,fps=param['fps'],isprint=False,datasets=['LGI-PPGI','MR-NIRP'],split=4,data_bpm=data_bpm)

### Show the result of evaluation

In [92]:
# Create empty nested dictionaries to store the sorted data
data_by_dataset = nested_dict(3, list)      # Data sorted by dataset and metric
data_by_act = nested_dict(3, list)          # Data sorted by activity and metric
data_by_group_act = nested_dict(3, list)    # Data sorted by group activity and metric
data_by_group_act_lgi = nested_dict(3, list)   
data_by_group_act_pure = nested_dict(3, list)   

# Loop through each dataset, name, activity, method, and metric in data_results
for dataset in data_results.keys():
    for name in data_results[dataset].keys():
        for act in data_results[dataset][name].keys():
            for method in data_results[dataset][name][act].keys():
                for metric in data_results[dataset][name][act][method].keys():
                    
                    # Append the data for each metric to the corresponding nested dictionary
                    data_by_dataset['Overall'][metric][method].extend([data_results[dataset][name][act][method][metric]]) 
                    data_by_dataset[dataset][metric][method].extend([data_results[dataset][name][act][method][metric]])                                         
                    data_by_act[act][metric][method].extend([data_results[dataset][name][act][method][metric]]) 

                    if act=='resting' or act=='01' or act=='still':                    
                        data_by_group_act['Rest'][metric][method].extend([data_results[dataset][name][act][method][metric]]) 
                    if act=='talk' or act=='02' or act=='motion':                    
                        data_by_group_act['Talk'][metric][method].extend([data_results[dataset][name][act][method][metric]]) 
                    if act=='rotation' or act=='05' or act=='06':                    
                        data_by_group_act['Rotation'][metric][method].extend([data_results[dataset][name][act][method][metric]]) 
                    if act=='gym':                    
                        data_by_group_act['Gym'][metric][method].extend([data_results[dataset][name][act][method][metric]])       
                    if act=='03' or act=='04':                    
                        data_by_group_act['Translation'][metric][method].extend([data_results[dataset][name][act][method][metric]])     

                    if dataset=='LGI-PPGI':
                        if act=='resting' :                    
                            data_by_group_act_lgi['Rest'][metric][method].extend([data_results[dataset][name][act][method][metric]]) 
                        if act=='talk' :                    
                            data_by_group_act_lgi['Talk'][metric][method].extend([data_results[dataset][name][act][method][metric]]) 
                        if act=='rotation' :                    
                            data_by_group_act_lgi['Rotation'][metric][method].extend([data_results[dataset][name][act][method][metric]]) 
                        if act=='gym':                    
                            data_by_group_act_lgi['Gym'][metric][method].extend([data_results[dataset][name][act][method][metric]])  

                    if dataset=='PURE':
                        data_by_group_act_pure[act][metric][method].extend([data_results[dataset][name][act][method][metric]])   
    

In [93]:
prop = fm.FontProperties(fname=".\\other\\fonts\\times-ro.ttf") 
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.family'] = 'DejaVu Serif'
plt.rcParams['font.serif'] = [prop.get_name()]

In [94]:
mode = 'ML'
info_data_path = '.\\results\\info_ML\\'
tables_path = ".\\results\\tables_ML\\"
figures_path = ".\\results\\figures_ML\\"
methods = []
methods = [
                'cpu_GREEN',
                # 'cpu_RED',
                # 'cpu_BLUE',
                # 'cpu_PCA',
                # 'cpu_ICA',
                'cpu_LGI',
                'cpu_CHROM',
                'cpu_POS',
                # 'cpu_PBV',
                # 'cpu_OMIT',
                'GRGB'
                ]
methods += ['Model']
methods_names = [method[4:] if (method!='GRGB' and method!='Model') else method for method in methods]
# methods_names[-1] = 'Our Model'
metrics = ['DTW','r'] 
# metrics = ['DTW','r','RMSE','bpm_PPG_fft', 'bpm_PPG_acorr', 'bpm_PPG_welch', 'bpm_rPPG_fft', 'bpm_rPPG_acorr', 'bpm_rPPG_welch', 'PPG_rPPG_fft', 'PPG_rPPG_acorr', 'PPG_rPPG_welch']
datasets = ['PURE', 'LGI-PPGI', 'MR-NIRP']
activity_groups = ['Rest', 'Talk', 'Translation', 'Rotation', 'Gym']
max_color = ['purple','darkblue','darkgreen']
other_color = ['purple','lightblue','lightgreen']


#### By dataset

In [95]:
# Get the results for the three channels
table_datasets, table_datasets_avg = get_table_results(data_by_dataset,datasets,metrics,methods,methods_names[:-1] + ['Our Model'])
table_datasets_avg = table_datasets_avg.round(2)
table_datasets_avg.to_csv(tables_path+'datasets_'+mode+'.csv')#, mode='a')
table_datasets_avg

DTW     r
PURE     GREEN      2.88  0.66
         LGI        2.30  0.72
         CHROM      1.72  0.73
         POS        2.28  0.72
         GRGB       2.59  0.69
         Our Model  1.17  0.83
LGI-PPGI GREEN      3.20  0.40
         LGI        2.89  0.48
         CHROM      2.69  0.47
         POS        2.84  0.51
         GRGB       2.98  0.43
         Our Model  1.65  0.49
MR-NIRP  GREEN      3.40  0.22
         LGI        2.97  0.21
         CHROM      2.98  0.22
         POS        2.87  0.23
         GRGB       3.10  0.20
         Our Model  1.73  0.24

In [96]:
# get the p-values 
if mode == 'ML':
    datasets_pvalues = get_pvalues_ML(data_by_dataset,datasets,metrics,methods_names[:-1] + ['Our Model'],table_datasets)
else:
    datasets_pvalues = get_pvalues_RGB(data_by_dataset,datasets,metrics,methods_names,table_datasets)
datasets_pvalues = datasets_pvalues.round(3)
datasets_pvalues.to_csv(tables_path+'datasets_pvalue_'+mode+'.csv')#, mode='a')
datasets_pvalues

DTW      r
PURE     GREEN vs. Our Model  0.001  0.001
         LGI vs. Our Model    0.001  0.001
         CHROM vs. Our Model  0.160  0.010
         POS vs. Our Model    0.001  0.001
         GRGB vs. Our Model   0.001  0.001
LGI-PPGI GREEN vs. Our Model  0.001  0.001
         LGI vs. Our Model    0.001  0.900
         CHROM vs. Our Model  0.009  0.900
         POS vs. Our Model    0.003  0.811
         GRGB vs. Our Model   0.001  0.007
MR-NIRP  GREEN vs. Our Model  0.001  0.664
         LGI vs. Our Model    0.006  0.263
         CHROM vs. Our Model  0.001  0.664
         POS vs. Our Model    0.114  0.900
         GRGB vs. Our Model   0.001  0.008

#### By activity

In [97]:
table_acts, table_acts_avg = get_table_results(data_by_group_act,activity_groups,metrics,methods,methods_names[:-1] + ['Our Model'])
table_datasets_avg = table_datasets_avg.round(2)
table_acts_avg.to_csv(tables_path+'activities_'+mode+'.csv')#, mode='a'
table_acts_avg

DTW     r
Rest        GREEN      2.82  0.52
            LGI        2.53  0.52
            CHROM      2.20  0.52
            POS        2.46  0.53
            GRGB       2.69  0.50
            Our Model  1.45  0.57
Talk        GREEN      3.28  0.34
            LGI        2.83  0.40
            CHROM      2.68  0.42
            POS        2.81  0.42
            GRGB       3.02  0.38
            Our Model  1.58  0.46
Translation GREEN      2.56  0.72
            LGI        2.20  0.74
            CHROM      1.58  0.75
            POS        2.20  0.74
            GRGB       2.46  0.72
            Our Model  1.20  0.84
Rotation    GREEN      3.38  0.59
            LGI        2.56  0.69
            CHROM      2.03  0.71
            POS        2.50  0.71
            GRGB       2.86  0.65
            Our Model  1.18  0.77
Gym         GREEN      3.17  0.24
            LGI        2.80  0.32
            CHROM      2.83  0.28
            POS        2.79  0.42
            GRGB       2.80  0.31
            Our Model  2.02  0.40

In [98]:
if mode == 'ML':
    acts_pvalues = get_pvalues_ML(data_by_group_act,activity_groups,metrics,methods_names[:-1] + ['Our Model'],table_acts)
else:
    acts_pvalues = get_pvalues_RGB(data_by_group_act,activity_groups,metrics,methods_names,table_acts)
acts_pvalues = acts_pvalues.round(3)
acts_pvalues.to_csv(tables_path+'act_pvalue_'+mode+'.csv')#, mode='a'
acts_pvalues

DTW      r
Rest        GREEN vs. Our Model  0.001  0.118
            LGI vs. Our Model    0.001  0.169
            CHROM vs. Our Model  0.064  0.459
            POS vs. Our Model    0.004  0.900
            GRGB vs. Our Model   0.001  0.001
Talk        GREEN vs. Our Model  0.001  0.001
            LGI vs. Our Model    0.001  0.009
            CHROM vs. Our Model  0.007  0.652
            POS vs. Our Model    0.004  0.900
            GRGB vs. Our Model   0.001  0.001
Translation GREEN vs. Our Model  0.001  0.001
            LGI vs. Our Model    0.002  0.009
            CHROM vs. Our Model  0.892  0.268
            POS vs. Our Model    0.004  0.013
            GRGB vs. Our Model   0.001  0.001
Rotation    GREEN vs. Our Model  0.001  0.001
            LGI vs. Our Model    0.001  0.009
            CHROM vs. Our Model  0.257  0.900
            POS vs. Our Model    0.001  0.159
            GRGB vs. Our Model   0.001  0.001
Gym         GREEN vs. Our Model  0.002  0.030
            LGI vs. Our Model    0.410  0.745
            CHROM vs. Our Model  0.207  0.087
            POS vs. Our Model    0.636  0.900
            GRGB vs. Our Model   0.410  0.410